In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Business Problem

## Situation: 
I just finished my bachelors degree and I have 4 job offers in Manila, I’m currently living in Abra which is around 8 hours drive to Manila. Provincial Life is very different from City life therefore before I choose which job should I pick I want to be able to determine which workplace is accessible to different amenities I require in order to survive. This project can be usefull to anyone who has never been to a place and would like a place with accessible amenities.

### Data Approach

I only have the location of the 4 workplaces, in order for me to decide which workplace is the best I will be using 4 square to get the nearby places where I can stay and will run foursquare again on this establishments to see the accessible/nearby amenities.

### workplace with location

In [2]:
work1 = {'lat' : '14.57082',
        'long' : '121.048411'}
work2 = {'lat' : '14.607486',
        'long' : '121.079501'}
work3 = {'lat' : '14.555066',
        'long' : '121.018639'}
work4 = {'lat' : '14.535043',
        'long' : '120.982712'}